# Part I. ETL Pipeline for Pre-Processing the Files

## Input Preprocessing

#### Import Python packages 

In [27]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [28]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    print("number of input files: ", len(file_path_list))

/home/claudiordgz/udacity/dend/project_2
number of input files:  30


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [29]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print("Total number of rows: ", len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


Total number of rows:  8056


In [30]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# CSV to Cassandra DB 

## <font color=red>event_datafile_new.csv</font> has been generated, and contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Cluster connection

#### Creating a Cluster

In [31]:
from cassandra import cluster

try: 
    cluster = cassandra.cluster.Cluster(['127.0.0.1']) #If you have a locally installed Apache Cassandra instance
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [32]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [33]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

# Table Creation

## Sample Queries

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

`select artist, song, length in music_app_history where sessionId=338 and itemInSession=4`

### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

`select artist, song, user_name in music_app_history where sessionId=182 and userid=10`

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

`select user_name in music_app_history where song='All Hands Against His Own'`


In [39]:
tables = [
    # Query 1: The PRIMARY KEY is based on our query WHERE clause
    """CREATE TABLE IF NOT EXISTS session_history 
    (session_id int,
     item_in_session int,
     artist text,
     song text,
     length float,
     PRIMARY KEY (
         session_id,
         item_in_session
    ))""",
    # Query 2: The PRIMARY KEY must be sorted by item_in_session, so we add that as a clustering column placed last
    # Because our query will only be using user_id and session_id, in that respective order
    """CREATE TABLE IF NOT EXISTS song_playlist_session 
    (session_id int,
     item_in_session int,
     user_id int,
     artist text,
     song text,
     user_name text,
     PRIMARY KEY (
         (
            user_id,
            session_id
         ),
         item_in_session
    ))""",
    # Query 3: We will be using song to query our Table, but the user_id is a better candidate for uniqueness than user_name
    # So we use both these properties for the PRIMARY KEY
    """CREATE TABLE IF NOT EXISTS user_song_history 
    (song text,
     user_id int,
     user_name text,
     PRIMARY KEY (
         song,
         user_id
    ))""",
]

for table in tables:
    try:
        session.execute(table)
    except Exception as e:
        print(e)

In [40]:
def to_val(key, line):
    """Will fetch the index in the row values and cast it if necessary.
    
    Keyword arguments:
    key -- the desired key based on the DB Schema
    line -- the line of values received from the CSV
    """
    indices = {
        "artist": 0,
        "first_name": 1,
        "gender": 2,
        "item_in_session": 3,
        "last_name": 4,
        "length": 5,
        "level": 6,
        "location": 7,
        "session_id": 8,
        "song": 9,
        "user_id": 10
    }
    value = line[indices[key]]
    casts = {
        "length": float,
        "session_id": int,
        "item_in_session": int,
        "user_id": int
    }
    if key in casts:
        value = casts[key](value)
    return value

def insert_to_cassandra(query, values):
    """Executes query and raises error if it was not possible
    
    Keyword arguments:
    query -- INSERT statement
    values -- tuple of values (1,2,3,4,...,n) expected by query
    """
    try:
        session.execute(query, values)
    except Exception as e:
        print(e)
        raise e


def insert_data_for_q1(row):
    """From an Array of values of event_datafile_new.csv
       will transform into proper input and insert to cassandra
       the following.
       (session_id int, item_in_session int, artist string, song string, length float)

    Keyword arguments:
    row -- row array of values received from CSV
    """
    values = (to_val("session_id", row), to_val("item_in_session", row), to_val("artist", row), to_val("song", row), to_val("length", row))
    query = "INSERT INTO session_history (session_id, item_in_session, artist, song, length)"
    query = query + " VALUES (%s, %s, %s, %s, %s)"
    insert_to_cassandra(query, values)


def insert_data_for_q2(row):
    """From an Array of values of event_datafile_new.csv
       will transform into proper input and insert to cassandra
       the following.
       (session_id int, item_in_session int, user_id int, artist string, song string, user_name string)

    Keyword arguments:
    row -- row array of values received from CSV
    """
    user_name = "{} {}".format(to_val("first_name", row), to_val("last_name", row))
    values = (to_val("session_id", row), to_val("item_in_session", row), to_val("user_id", row), to_val("artist", row), to_val("song", row), user_name)
    query = "INSERT INTO song_playlist_session (session_id, item_in_session, user_id, artist, song, user_name)"
    query = query + " VALUES (%s, %s, %s, %s, %s, %s)"
    insert_to_cassandra(query, values)


def insert_data_for_q3(row):
    """From an Array of values of event_datafile_new.csv
       will transform into proper input and insert to cassandra
       the following.
       (song string, user_name string, user_id int)

    Keyword arguments:
    row -- row array of values received from CSV
    """
    user_name = "{} {}".format(to_val("first_name", row), to_val("last_name", row))
    values = (to_val("song", row), to_val("user_id", row), user_name)
    query = "INSERT INTO user_song_history (song, user_id, user_name)"
    query = query + " VALUES (%s, %s, %s)"
    insert_to_cassandra(query, values)


In [41]:
# Load CSV data
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        insert_data_for_q1(line)
        insert_data_for_q2(line)
        insert_data_for_q3(line)

#### SELECT to verify that the data have been inserted into each table

In [45]:
## Add in the SELECT statement to verify the data was entered into the table

tables = ["session_history", "song_playlist_session", "user_song_history"]
columns = [("artist", "song", "length"), ("artist", "song", "user_name"), ("user_name", "song")]
for col, t in zip(columns, tables):
    print("GETTING RECORDS FROM TABLE: {}".format(t))
    query = "select {} from {} limit 5;".format(",".join(col), t)
    try:
        rows = session.execute(query)
        df = pd.DataFrame(rows, columns=col)
        display(df)
    except Exception as e:
        print(e)
    print()

GETTING RECORDS FROM TABLE: session_history


,artist,song,length
0,Regina Spektor,The Calculation (Album Version),191.085266
1,Octopus Project,All Of The Champs That Ever Lived,250.957916
2,Tegan And Sara,So Jealous,180.061584
3,Dragonette,Okay Dolores,153.390564
4,Lil Wayne / Eminem,Drop The World,229.589752



GETTING RECORDS FROM TABLE: song_playlist_session


,artist,song,user_name
0,System of a Down,Sad Statue,Emily Benson
1,Ghostland Observatory,Stranger Lover,Emily Benson
2,Evergreen Terrace,Zero,Emily Benson
3,Deftones,Head Up (LP Version),Kinsley Young
4,The Notorious B.I.G.,Playa Hater (Amended Version),Kinsley Young



GETTING RECORDS FROM TABLE: user_song_history


,user_name,song
0,Chloe Cuevas,Wonder What's Next
1,Chloe Cuevas,In The Dragon's Den
2,Aleena Kirby,Too Tough (1994 Digital Remaster)
3,Chloe Cuevas,Rio De Janeiro Blue (Album Version)
4,Lily Koch,My Place


In [46]:
## Query 1

query = "select artist, song, length from session_history where session_id=338 and item_in_session=4"
columns = ("artist", "song", "length")
try:
    rows = session.execute(query)
    df = pd.DataFrame(rows, columns=columns)
    display(df)
except Exception as e:
    print(e)
    

,artist,song,length
0,Faithless,Music Matters (Mark Knight Dub),495.307312


In [47]:
## Query 2 Ordered by itemInSession

query = "select artist, song, user_name from song_playlist_session where session_id=182 and user_id=10"
columns = ("artist", "song", "user_name")
try:
    rows = session.execute(query)
    df = pd.DataFrame(rows, columns=columns)
    display(df)
except Exception as e:
    print(e)

                    

,artist,song,user_name
0,Down To The Bone,Keep On Keepin' On,Sylvie Cruz
1,Three Drives,Greece 2000,Sylvie Cruz
2,Sebastien Tellier,Kilometer,Sylvie Cruz
3,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie Cruz


In [48]:
## Query 3

query = "select user_name from user_song_history where song='All Hands Against His Own'"
columns = ("user_name",)
try:
    rows = session.execute(query)
    df = pd.DataFrame(rows, columns=columns)
    display(df)
except Exception as e:
    print(e)

,user_name
0,Jacqueline Lynch
1,Tegan Levine
2,Sara Johnson


### Drop the tables before closing out the sessions

In [49]:
tables = ["session_history", "song_playlist_session", "user_song_history"]
for t in tables:
    try:
        session.execute("drop table {}".format(t))
    except Exception as e:
        print(e)

### Close the session and cluster connection¶

In [50]:
session.shutdown()
cluster.shutdown()